#Web Scraping


In [1]:
import requests
import json
from bs4 import BeautifulSoup


In [2]:
page_url = 'https://www.pudelek.pl/widzowie-amerykanskiego-mam-talent-zachwyceni-finalowym-wystepem-sary-james-jest-gotowa-na-podboj-swiata-dla-mnie-juz-wygrala-6812198149712640a/'
page = requests.get(page_url)

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="pl">
 <head>
  <meta charset="utf-8" class="next-head"/>
  <link class="next-head" href="/static/favicon.ico" rel="shortcut icon"/>
  <link class="next-head" href="/static/opensearch.xml" rel="search" title="Pudelek" type="application/opensearchdescription+xml"/>
  <link class="next-head" href="/static/apple-touch-icon-60x60.png" rel="apple-touch-icon"/>
  <link class="next-head" href="/static/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link class="next-head" href="/static/apple-touch-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
  <link class="next-head" href="/static/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link class="next-head" href="/static/apple-touch-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
  <link class="next-head" href="/static/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link class="next-head" href="/static/apple-touch-icon-120x120.png" re

In [10]:
clas = "sc-1mskw74-0 sc-7eqdwf-0 sc-q1w81m-0 cyNurK"
comm = soup.find_all("div", {"class": clas})
print(comm[4].get_text())
print(len(comm))

Ludzie nie wysyłają esów na tych co są nadmiernie chwaleni wychodząc z założenia, że i tak inni wyślą...
19


**HTML file does not contain all comments, padding is inteactive and does not chcenge the url adress, so there is a need to use API through GraphQL.**

In [24]:
#first 5 comments
query = """query comments(
    $productId: String!
      $materialId: String!
        $parentId: String
          $offset: Int = 0
            $first: Int = 10
              $sort: CommentsSortType = LATEST
               
                  
                  ) {
                      comments: comments(
                            productId: $productId
                                contentId: $materialId
                                    parentId: $parentId
                                        offset: $offset
                                            first: $first
                                                sort: $sort
                                                  ) {
                                                        ...commentEdges
                                                            pageInfo {
                                                                    hasNextPage
                                                                          hasPreviousPage
                                                                                endCursor
                                                                                      totalCount
                                                                                          }
                                                                                            }

                                                                                                                                  }
                                                                                                                                  
                                                                                                                                  fragment commentEdges on CommentsConnection {
                                                                                                                                      edges {
                                                                                                                                            node {
                                                                                                                                                        text
                                                                                                                                                                nick

                                                                                                                                                                                                  }
                                                                                                                                                                                                    }
                                                                                                                                                                                                      }"""

#query = "query":"query comments(\n  $productId: String!\n  $materialId: String!\n  $parentId: String\n  $offset: Int = 0\n  $first: Int = 10\n  $sort: CommentsSortType = LATEST\n  $popular: Boolean = false\n  $popularFirst: Int = 3\n) {\n  comments: comments(\n    productId: $productId\n    contentId: $materialId\n    parentId: $parentId\n    offset: $offset\n    first: $first\n    sort: $sort\n  ) {\n    ...commentEdges\n    pageInfo {\n      hasNextPage\n      hasPreviousPage\n      endCursor\n      totalCount\n    }\n  }\n  popular: comments(\n    productId: $productId\n    contentId: $materialId\n    parentId: $parentId\n    offset: 0\n    first: $popularFirst\n    sort: TOP_RATED\n  ) @include(if: $popular) {\n    ...commentEdges\n  }\n}\n\nfragment commentEdges on CommentsConnection {\n  edges {\n    node {\n      id\n      text\n      nick\n      created\n      modified\n      votesUp\n      votesDown\n      replies\n    }\n  }\n}\n"                                                                                                                                                                                                   
data = {"query": query, "variables":{"productId":"5980322589471361","materialId":"6812198149712640","offset":0,"first":99,"sort":"LATEST"}}


response = requests.post('https://data-api.wp.pl/graphql', json=data)
print(json.dumps(response.json(), indent=2))

{
  "data": {
    "comments": {
      "edges": [
        {
          "node": {
            "text": "\u015arednio ",
            "nick": "SUPERMAN"
          }
        },
        {
          "node": {
            "text": "Cudowny g\u0142os, wspania\u0142a dziewczyna. \u263a\ufe0f\nTylko, \u017ceby nie sko\u0144czy\u0142a jak Whitney, Amy i pozostali. ",
            "nick": "NICOLE"
          }
        },
        {
          "node": {
            "text": "Sorry ale po otrzymaniu wykszta\u0142cenia muzycznego mog\u0119 \u015bmia\u0142o powiedzie\u0107 ze dziewczyna fa\u0142szowa\u0142a wiele razy podczas tego nagrania...Stare powiedzenie Wojew\u00f3dzkiego &#34;9/10 g\u0142uchych a jeden zle s\u0142yszy&#34; okazuje si\u0119 adekwatne nie tylko do Polski, ale i ca\u0142ego \u015bwiata...",
            "nick": "nie_moja_bajka"
          }
        },
        {
          "node": {
            "text": "Caly czas tylko smutne piosenki spiewa",
            "nick": "Zachary "
          }
       

In [14]:
soup = BeautifulSoup(response.content, 'html.parser') #to get Polish letters

print(soup.prettify())

{"data":{"comments":{"edges":[{"node":{"text":"Jest bardzo dobra. Zasługuje na zwycięstwo. Prawdziwy talent, powodzenia 👍","nick":"Aga"}},{"node":{"text":"To tylko turniej tv mielący tysiące coverów. ","nick":"bezhejtu"}},{"node":{"text":"Zepsuła ta piosenką wszystko wogole nie było tego klimatu co przy Kate Bush głos Sary nie pasuje do tego utworu ","nick":"Adam"}},{"node":{"text":"Talent jest ale trzeba też przebojów. Teraz z tym krucho.","nick":"gość"}},{"node":{"text":"To był jej najgorszy występ kto jej wybrał ta piosenke przecież ona ja zepsuła ","nick":"Beata"}},{"node":{"text":"Akurat mnie nie podobało się chociaż bardzo lubię SARE I TEZ uważam że piękne śpiewa ale akurat z tą piosenką coś poszło nie tak, oryginał sto razy lepszy,\n\n","nick":"Sylwia"}},{"node":{"text":"Nie podniecajcie się jeszcze nie wygrała a konkurencja jest silna ","nick":"Basia"}},{"node":{"text":"Ciekawe czy Sara napisałaby sama taką piosenkę?","nick":"Lex"}},{"node":{"text":"Najwyraźniej zadziałał syndr

In [25]:
query = "query comments(\n  $productId: String!\n  $materialId: String!\n  $parentId: String\n  $offset: Int = 0\n  $first: Int = 10\n  $sort: CommentsSortType = LATEST\n  $popular: Boolean = false\n  $popularFirst: Int = 3\n) {\n  comments: comments(\n    productId: $productId\n    contentId: $materialId\n    parentId: $parentId\n    offset: $offset\n    first: $first\n    sort: $sort\n  ) {\n    ...commentEdges\n    pageInfo {\n      hasNextPage\n      hasPreviousPage\n      endCursor\n      totalCount\n    }\n  }\n  popular: comments(\n    productId: $productId\n    contentId: $materialId\n    parentId: $parentId\n    offset: 0\n    first: $popularFirst\n    sort: TOP_RATED\n  ) @include(if: $popular) {\n    ...commentEdges\n  }\n}\n\nfragment commentEdges on CommentsConnection {\n  edges {\n    node {\n      id\n      text\n      nick\n      created\n      modified\n      votesUp\n      votesDown\n      replies\n    }\n  }\n}\n"                                                                                                                                                                                                   
data = {"query": query, "variables":{"productId":"5980322589471361","materialId":"6812198149712640","parentId":"6812211080173697","offset":0,"first":99,"sort":"LATEST"}}


response = requests.post('https://data-api.wp.pl/graphql', json=data)
print(json.dumps(response.json(), indent=2))

{
  "data": {
    "comments": {
      "edges": [
        {
          "node": {
            "id": "6812246114961537",
            "text": "Naturalnym jest,ze nie wszystkim przypad\u0142 wyst\u0119p do gustu :-) Tyle ile ludzi na tym \u015bwiecie tyle opinii i upodoba\u0144 \ud83d\ude09",
            "nick": "Marie ",
            "created": 1663145931,
            "modified": 1663145931,
            "votesUp": 7,
            "votesDown": 1,
            "replies": 0
          }
        },
        {
          "node": {
            "id": "6812234302023810",
            "text": "Nie wszyscy sa zachwyceni. Owszem jest fajna i ma piekny glos, ale jej poprzednie piosenki byly lepsze. Teraz ten odsloniety brzuchol i wycie-nie wiem do teraz jaka piosenke zaspiewala tak ja przerobila. Ta z australii wygrala w swoim kraju, drake sie nie postaral spiewajac to samo, chapel falszowaly. Powinny wygrac mayyas",
            "nick": "Lola",
            "created": 1663143048,
            "modified": 166314

get id when replies >0 

https://www.pudelek.pl/widzowie-amerykanskiego-mam-talent-zachwyceni-finalowym-wystepem-sary-james-jest-gotowa-na-podboj-swiata-dla-mnie-juz-wygrala-6812198149712640a